## Приведён пример генерации, для сравнения с основным методом

In [1]:
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from rdkit import Chem
from rdkit.Chem import Descriptors, QED, Lipinski, AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogParams
from rdkit.Chem import rdMolDescriptors

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, set_seed
from tqdm import tqdm
#import selfies as sf

import warnings
warnings.filterwarnings('ignore')

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.
/home/elizaveta/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-15 23:28:56.859991: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752611336.924619 3753792 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752611336.943927 3753792 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0

### Генерация молекул (DrugGPT)

In [2]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

tokenizer_path = "tokenizer.json"
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_path)
tokenizer.pad_token = "<pad>"
tokenizer.bos_token = "<bos>"
tokenizer.eos_token = "<eos>"

In [3]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained("jonghyunlee/MolGPT_pretrained-by-ZINC15")

In [4]:
def generate_smiles(model, tokenizer, num_sequences=1000, temperature=1.0, seed=42):
    set_seed(seed)
    torch.manual_seed(seed)
    
    outputs = model.generate(
        max_length=128,
        num_return_sequences=num_sequences,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=temperature,
        return_dict_in_generate=True
    )
    
    smiles_list = []
    for seq in outputs.sequences:
        smile = tokenizer.decode(seq, skip_special_tokens=True)
        if smile and smile.count('C') > 0:
            smiles_list.append(smile)
    
    return smiles_list

In [5]:
generated_smiles = generate_smiles(model, tokenizer, num_sequences=1000, temperature=0.7, seed=42)

In [6]:
generated_smiles = pd.DataFrame(generated_smiles, columns=['Smiles'])
generated_smiles.shape[0]

1000

### Проверка сгенерированных SMILES на валидность

In [7]:
def is_valid_smiles(smiles):
    if pd.isna(smiles):
        return False
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None

In [8]:
generated_smiles['is_valid'] = generated_smiles['Smiles'].apply(is_valid_smiles)
generated_smiles[~generated_smiles['is_valid']].shape[0]

0

Как видно, проблемой MolGPT является невалидность молекул